# 单线程


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import openpyxl

# ...

# 获取基因功能内容
def get_gene_function(url):
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        gene_function = driver.find_element(By.XPATH, '//section[2]/div[1]/ul/li/p').text
        driver.quit()
        return gene_function.strip()
    except:
        return 'NA'

# ...

if __name__ == '__main__':
    input_txt_path = 'gene_list10.txt'
    output_excel_path = 'gene_list10.xlsx'  # 设置保存结果的 Excel 文件路径

    with open(input_txt_path, 'r') as file:
        gene_list = [line.strip() for line in file]

    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.cell(row=1, column=1, value='Gene Name')
    worksheet.cell(row=1, column=2, value='Has Label')

    for row, gene_name in enumerate(gene_list, start=2):
        time.sleep(1)  # 限制请求频率

        new_url = f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={gene_name}'

        gene_function = get_gene_function(new_url)

        labels_to_check = ["receptor", "ligand", "receptors", "ligands", "cell-cell adhesion",
                           "intercellular interaction"]
        has_label = any(label in gene_function.lower() for label in labels_to_check)

        worksheet.cell(row=row, column=1, value=gene_name)
        worksheet.cell(row=row, column=2, value=1 if has_label else 0)

        print(f"Gene {gene_name} processed")

    workbook.save(output_excel_path)
    print(f"Results saved to '{output_excel_path}'")


# 多线程

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import threading
import openpyxl

# ...

# 获取基因功能内容
def get_gene_function(url):
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        gene_function = driver.find_element(By.XPATH, '//section[2]/div[1]/ul/li/p').text
        driver.quit()
        return gene_function.strip()
    except:
        return 'NA'

# ...

def process_gene_chunk(gene_chunk, worksheet, start_row):
    for i, gene_name in enumerate(gene_chunk, start=start_row):
        time.sleep(1)  # 限制请求频率

        new_url = f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={gene_name}'

        gene_function = get_gene_function(new_url)

        labels_to_check = ["receptor", "ligand", "receptors", "ligands", "cell-cell adhesion",
                           "intercellular interaction"]
        has_label = any(label in gene_function.lower() for label in labels_to_check)

        worksheet.cell(row=i, column=1, value=gene_name)
        worksheet.cell(row=i, column=2, value=1 if has_label else 0)


if __name__ == '__main__':
    input_txt_path = 'seqfish_M3drop9.txt'
    output_excel_path = 'seqfish_M3drop9.xlsx'

    with open(input_txt_path, 'r') as file:
        gene_list = [line.strip() for line in file]

    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.cell(row=1, column=1, value='Gene Name')
    worksheet.cell(row=1, column=2, value='Has Label')

    chunk_size = 200  # 每一百个基因一个线程
    threads = []

    for chunk_start in range(0, len(gene_list), chunk_size):
        gene_chunk = gene_list[chunk_start:chunk_start + chunk_size]
        thread = threading.Thread(target=process_gene_chunk, args=(gene_chunk, worksheet, chunk_start + 2))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    workbook.save(output_excel_path)
    print(f"Results saved to '{output_excel_path}'")


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# 设置base_url是为了得到url的cookies，网站设置了反爬机制
base_url = "https://www.genecards.org"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0',
}

# 获取cookies
def get_cookies(url):
    try:
        sessions = requests.get(url, headers=headers)
        cookie = sessions.cookies
    except:
        cookie = ''
    return cookie

# 获取基因是否包含指定标签
def has_any_label(url, cookies, labels):
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        content = response.content.decode()
        data = BeautifulSoup(content, 'html.parser')
        gene_content = data.find('section', id='summaries').get_text()
        return int(any(label.lower() in gene_content.lower() for label in labels))
    except:
        return 0

# 主程序
if __name__ == '__main__':
    input_txt_path = 'gene_names.txt'
    output_txt_path = 'gene_label_results.txt'
    labels_to_check = ["receptor", "ligand", "receptors", "ligands", "cell-cell adhesion", "intercellular interaction"]
    
    with open(input_txt_path, 'r') as file:
        gene_list = [line.strip() for line in file]
    
    with open(output_txt_path, 'w') as output_file:
        for gene_name in gene_list:
            time.sleep(1)  # 限制请求频率
            
            new_url = f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={gene_name}'
            cookies = get_cookies(base_url)
            
            has_label = has_any_label(new_url, cookies, labels_to_check)
            
            result = f"Gene {gene_name} has any label: {has_label}\n"
            output_file.write(result)
            print(result)


Gene Gene has any label: 0

Gene Cplx1 has any label: 0

Gene Slc17a7 has any label: 0

Gene Eef1a2 has any label: 0

Gene Clstn1 has any label: 0

Gene Kif5a has any label: 0

Gene Mag has any label: 0

Gene Cadm4 has any label: 0

Gene Lynx1 has any label: 0

Gene Cnp has any label: 0

Gene Shank1 has any label: 0

Gene Sptbn2 has any label: 0

Gene Tubb4a has any label: 0

Gene Dynll2 has any label: 0

Gene Arc has any label: 0

Gene Lingo1 has any label: 0

Gene Nptxr has any label: 0

Gene Syngap1 has any label: 0

Gene Vgf has any label: 0

Gene Agap2 has any label: 0

Gene Gsn has any label: 0

Gene Grin1 has any label: 0

Gene Cend1 has any label: 0

Gene Cldn11 has any label: 0

Gene Efhd2 has any label: 0

Gene Atp1a1 has any label: 0

Gene Tmem132a has any label: 0

Gene Kcnc1 has any label: 0

Gene Rprm has any label: 0

Gene Mapk8ip1 has any label: 0

Gene Wbp2 has any label: 0

Gene Ngef has any label: 0

Gene Atp1b2 has any label: 0

Gene Slc6a17 has any label: 0

Gene S

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# 设置base_url是为了得到url的cookies，网站设置了反爬机制
base_url = "https://www.genecards.org"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0',
}

# 获取cookies
def get_cookies(url):
    try:
        sessions = requests.get(url, headers=headers)
        cookie = sessions.cookies
    except:
        cookie = ''
    return cookie

# 获取基因功能内容
def get_gene_function(url, cookies):
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        content = response.content.decode()
        data = BeautifulSoup(content, 'html.parser')
        gene_function = data.find('section', id='summaries').get_text()
        return gene_function.strip()
    except:
        return 'NA'

# 主程序
if __name__ == '__main__':
    input_txt_path = 'gene_names.txt'
    
    with open(input_txt_path, 'r') as file:
        gene_list = [line.strip() for line in file]
    
    for gene_name in gene_list:
        time.sleep(1)  # 限制请求频率
        
        new_url = f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={gene_name}'
        cookies = get_cookies(base_url)
        
        gene_function = get_gene_function(new_url, cookies)
        
        print(f"Gene {gene_name} function:\n{gene_function}\n")


Gene Gene function:
NA

Gene Cplx1 function:
NA

Gene Slc17a7 function:
NA

Gene Eef1a2 function:
NA

Gene Clstn1 function:
NA

Gene Kif5a function:
NA

Gene Mag function:
NA

Gene Cadm4 function:
NA



In [1]:
def search(gene):
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.action_chains import ActionChains
    driver = webdriver.Chrome()
    web = driver.get('https://www.genecards.org/cgi-bin/carddisp.pl?gene='+gene)
    gene_function= driver.find_element('xpath','//section[2]/div[1]/ul/li/p')
    # for item in gene_function:
    print(gene+':'+gene_function.text)
    driver.quit()
if __name__== '__main__':
    fp = open("gene_list.txt",mode="r",encoding="utf-8")
    genes = fp.read().splitlines()
    for gene in genes:
        # print(gene)
        search(gene)
        






WebDriverException: Message: unknown error: Chrome failed to start: crashed.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/tjzhang01/.cache/selenium/chrome/linux64/116.0.5845.96/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x561c3ce97e23 <unknown>
#1 0x561c3cbc07a7 <unknown>
#2 0x561c3cbeb50a <unknown>
#3 0x561c3cbe8a25 <unknown>
#4 0x561c3cc28d92 <unknown>
#5 0x561c3cc283af <unknown>
#6 0x561c3cc1fde3 <unknown>
#7 0x561c3cbf3a7b <unknown>
#8 0x561c3cbf481e <unknown>
#9 0x561c3ce59638 <unknown>
#10 0x561c3ce5d507 <unknown>
#11 0x561c3ce67c4c <unknown>
#12 0x561c3ce5e136 <unknown>
#13 0x561c3ce2c9cf <unknown>
#14 0x561c3ce81b98 <unknown>
#15 0x561c3ce81d68 <unknown>
#16 0x561c3ce90cb3 <unknown>
#17 0x7fccc45f9ea5 start_thread


NameError: name 'PACSIN3' is not defined